In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [4]:
import numpy as np
import datetime
import os
import copy

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"
Label = ['LeftHand','RightHand','LeftLeg','Rightleg','West','Chest']

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"
StorePath = "/home/takeyama/Documents/"

if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+DATE+'window/'): 
    os.makedirs(StorePath+TITLE+DATE+'window/')
WindowDataPath=StorePath+TITLE+DATE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+TITLE+DATE+'graph/')
GraphPath=StorePath+TITLE+DATE+'graph/'

**センサデータの読み込み**

In [5]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する．

In [6]:
def ImportCSV(csv_file,SensorName,mode='Round'):
    import pandas as pd
    '''
    ~Argument~
    csv_file -> ファイル名 
    mode Round -> 四捨五入
         Roundup -> 切り上げ
         Rounddown -> 切り捨て
    
    ~Conversion~
    Acc Data  [0.1mG]=>[G]
    Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
    '''
    # data dictionary 
    RawData={}   
    AccConversion = 0.1 * 0.001
    GyrConversion = 0.01
    
    # design dataframe and import csv
    data = pd.read_csv(csv_file)
    data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
    data = data[ data['Type']=='ags']

    # convert numpy.darray 

    AccX=data.AccX.values*AccConversion
    AccY=data.AccY.values*AccConversion
    AccZ=data.AccZ.values*AccConversion
    
    GyrX=data.GyrX.values*GyrConversion
    GyrY=data.GyrY.values*GyrConversion
    GyrZ=data.GyrZ.values*GyrConversion

    # regist each raw data 
    RawData['AccX'] = AccX
    RawData['AccY'] = AccY
    RawData['AccZ'] = AccZ
    RawData['GyrX'] = GyrX
    RawData['GyrY'] = GyrY
    RawData['GyrZ'] = GyrZ
    RawData['Name'] = SensorName

    RawData['Time'] = data.Time.values

    return RawData

In [7]:
import processing
import window

Label = ['LeftHand','RightHand','LeftLeg','Rightleg','West','Chest']

DataFileNameList,SensorName=SensorDataFileNameList(DataPath)
l=[]
for i in range( len(DataFileNameList) ):
    l.append( ImportCSV(DataPath+DataFileNameList[i],Label[i]) )

In [8]:
def MakeCommonSection(inputDataArray):
    # 共通区間のスタート時間、ゴール時間を求める
    # スタート時間を求める
    startTime = min(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if startTime < min(inputDataArray[i]['Time']):
            startTime = min(inputDataArray[i]['Time'])
    
    # ゴール時間を求める
    goalTime = max(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if goalTime > max(inputDataArray[i]['Time']):
            goalTime = max(inputDataArray[i]['Time'])

    # 共通区間のスタート時間のインデックス、ゴール時間のインデックスを探索する
    def CalcSearchIndexFromTime(data, keyTime):
        """
        data　辞書型
        keyTime data['Time']の中の探す値
        """
        count = 0
        for i in range(0, len(data['Time'])):
            if keyTime == data['Time'][i]:
                print str(keyTime)+' is much in the index  whose number is '+str(i)
                return i
            
    startIndex = np.array([])
    goalIndex = np.array([])
    for obj in inputDataArray:
        print 'start'
        startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
        print 'goal'
        goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

    tmp={}
    comDataArray =[]
    key={}

    # センサデータすべて（時刻、加速度、角速度）に対して共通区間のみのデータを抽出
    for number,iDA in enumerate( inputDataArray ):

        tmp['AccX'] = copy.deepcopy( iDA['AccX'][startIndex[number]:goalIndex[number]] )
        tmp['AccY'] = copy.deepcopy( iDA['AccY'][startIndex[number]:goalIndex[number]] )
        tmp['AccZ'] = copy.deepcopy( iDA['AccZ'][startIndex[number]:goalIndex[number]] )
        tmp['GyrX'] = copy.deepcopy( iDA['GyrX'][startIndex[number]:goalIndex[number]] )
        tmp['GyrY'] = copy.deepcopy( iDA['GyrY'][startIndex[number]:goalIndex[number]] )
        tmp['GyrZ'] = copy.deepcopy( iDA['GyrZ'][startIndex[number]:goalIndex[number]] )
        tmp['Time'] = copy.deepcopy( iDA['Time'][startIndex[number]:goalIndex[number]] )
        tmp['Name'] = copy.deepcopy( iDA['Name'] )
        comDataArray.append(copy.deepcopy(tmp) )    
        key[ tmp['Name'] ] = number
       
    return key,comDataArray

In [9]:
TrimKey,TrimData = MakeCommonSection(l)

start
36290413 is much in the index  whose number is 35
goal
37368283 is much in the index  whose number is 1077905
start
36290413 is much in the index  whose number is 0
goal
37368283 is much in the index  whose number is 1077870
start
36290413 is much in the index  whose number is 959
goal
37368283 is much in the index  whose number is 1078829
start
36290413 is much in the index  whose number is 911
goal
37368283 is much in the index  whose number is 1078781
start
36290413 is much in the index  whose number is 871
goal
37368283 is much in the index  whose number is 1078741
start
36290413 is much in the index  whose number is 834
goal
37368283 is much in the index  whose number is 1078704


In [10]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt

# Generate WindowFrame Function
from mymodule import window

# timer
import time

%matplotlib inline

In [11]:
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils.visualize_util import model_to_dot, plot
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

import matplotlib.image as mpimg


Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5005)


In [12]:
def LoadDataFromNPZ(Path,DataName):
    """
    Path 　　NPZファイルがあるパス
    DataName 名前，ここでは装着した部位を指す
    """
    Label = ['LeftHand','RightHand','LeftLeg','RightLeg','West','Chest']
    
    # check which DataName is wrong
    if not DataName in Label: return -1
    
    for i in range( len(Label) ):
        if DataName == Label[i]:
            return np.load(Path+DataName+'.npz')['data'][()]

Variational AutoEncoderを実装してみる

In [38]:
from keras.datasets import mnist
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras import callbacks
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Lambda, merge
from keras.models import Model
from keras import backend as K
from keras.regularizers import activity_l2
from keras.utils.visualize_util import plot
from IPython.display import Image

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [31]:
# パラメータ
nb_epoch=30
batch_size=32
z_dim =4
epsilon_std = 1.0 # if epsilon is about 0, VAE is closing to AutoEncoder
latent_dim = 10

In [32]:
# 訓練データ
w=window()
w.SetData('AccX',TrimData[0]['AccX'])
wind=w.Compile(windowWidth=32,slidingWidth=16)
windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

  6%|▌         | 1886/33682 [00:00<00:01, 18851.30it/s]

AccX is registed now
Build Complete
Source shape is(1077870,)
window frames num =33682
offset=14


100%|██████████| 33682/33682 [00:17<00:00, 1947.80it/s]


In [33]:
def binary_crossentropy(y_true, y_pred):
    return K.sum(K.binary_crossentropy(y_pred, y_true), axis=-1)

def vae_loss(x, x_decoded_mean):
    reconst_loss = K.mean(binary_crossentropy(x, x_decoded_mean),axis=-1)
    latent_loss =  - 0.5 * K.mean(K.sum(1 + K.log(K.square(z_sigma)) - K.square(z_mean) - K.square(z_sigma), axis=-1))
    return reconst_loss + latent_loss

In [34]:
#Encoder Layer
#I change z_sigma_log to z_sigma from original. But it doesn't influcence the result
x = Input(shape=(windoW.shape[1] ,))
hidden = Dense(500, activation='tanh')(x)
z_mean = Dense(z_dim, activation='linear')(hidden)
z_sigma = Dense(z_dim, activation='linear')(hidden)

In [36]:
#Sampling Function
def sampling(args):
    z_mean, z_sigma = args
    epsilon = K.random_normal(shape=(z_dim,), mean=0., std=epsilon_std)
    return z_mean + z_sigma * epsilon

In [39]:
#Sampling
z = Lambda(sampling, output_shape=(z_dim,))([z_mean, z_sigma])

In [41]:
#Decoder Layer
decoder_h = Dense(500, activation='tanh')
decoder_mean = Dense(784, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [46]:
vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)

In [47]:
hist_vae = vae.fit(X_train, X_train,
        nb_epoch=nb_epoch,
        batch_size=batch_size,
        shuffle = True,              
        validation_data=(X_test, X_test))

NameError: name 'X_train' is not defined